In [ ]:
# HuggingFace関連のライブラリー
!pip install transformers[torch]
!pip install datasets
!pip install huggingface_hub
# トークナイザー用に日本語対応パッケージ
!pip install ipadic
!pip install fugashi
!pip install unidic_lite
# その他
# !pip install wandb  # transformersライブラリーのTrainerクラスのtrainメソッドを行う時のエラー対策
!pip install torchinfo

In [2]:
import numpy as np
import pandas as pd
import pprint
import torch
import torchinfo
import transformers
import datasets
from sklearn import metrics
from google.colab import userdata
from huggingface_hub import login

In [3]:
print("numpy：", np.__version__)
print("pandas：", pd.__version__)
print("torch：", torch.__version__)
print("transformers：", transformers.__version__)
print("datasets：", datasets.__version__)

numpy： 2.0.2
pandas： 2.2.2
torch： 2.6.0+cu124
transformers： 4.50.2
datasets： 3.5.0


In [4]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
HuggingFace_API_KEY = userdata.get("HuggingFace_API_KEY")

**テキスト分類を学習するための学習データを収集**

In [ ]:
# ネタになるデータをHuggingFaceから取得
dataset_dict = datasets.load_dataset("tyqiangz/multilingual-sentiments", "japanese")

In [6]:
# 取得したデータと、そのタイプを確認
print(dataset_dict)
print(type(dataset_dict))
# 取得したデータの項目を確認(trainを対象にして確認)
print("データの項目")
pprint.pprint(dataset_dict["train"].features)
# DatasetDictのtrainのデータの中身を2つ確認
print("データの中身")
pprint.pprint(dataset_dict["train"][0:2])
# 取得したデータ量が多いので、用いるデータは全体の1%だけにする
dataset_dict["train"] = dataset_dict["train"].select(indices=range(dataset_dict["train"].num_rows//1))
dataset_dict["validation"] = dataset_dict["validation"].select(indices=range(dataset_dict["validation"].num_rows//1))
dataset_dict["test"] = dataset_dict["test"].select(indices=range(dataset_dict["test"].num_rows//1))
# データ量を減らしたdataset_dictを確認
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 120000
    })
    validation: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 3000
    })
})
<class 'datasets.dataset_dict.DatasetDict'>
データの項目
{'label': ClassLabel(names=['positive', 'neutral', 'negative'], id=None),
 'source': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None)}
データの中身
{'label': [2, 2],
 'source': ['amazon_reviews_multi', 'amazon_reviews_multi'],
 'text': ['普段使いとバイクに乗るときのブーツ兼用として購入しました。見た目や履き心地は良いです。 '
          'しかし、２ヶ月履いたらゴム底が削れて無くなりました。また、バイクのシフトペダルとの摩擦で表皮が剥がれ、本革でないことが露呈しました。ちなみに防水とも書いていますが、雨の日は内部に水が染みます。 '
          '安くて見た目も良く、履きやすかったのですが、耐久性のなさ、本革でも防水でも無かったことが残念です。結局、本革の防水ブーツを買い直しました。',
          '十分な在庫を用意できない販売元も悪いですが、Amazonやら楽⚪︎が転売を認めちゃってるのが結果的に転売の後押しになっちゃってるんだよなぁ… '
          'Amazonもここぞとばかりに抱き合わせ販売しまくるし… それを恥ずかしいと思えなくな

DatasetDictをDataFrameに変換してcsvファイルで保存する(後続処理でデータをシャッフルするため)

In [ ]:
# DatasetDictをDataFrameに変換する準備
dataset_dict.set_format(type="pandas")
# それぞれtrainとvalidationとtestをcsvファイルに保存
dataset_dict["train"].to_csv(path_or_buf="./hugging_face_dataset_train.csv",
                             index=None)
dataset_dict["validation"].to_csv(path_or_buf="./hugging_face_dataset_validation.csv",
                                  index=None)
dataset_dict["test"].to_csv(path_or_buf="./hugging_face_dataset_test.csv",
                            index=None)
# データのタイプをDatasetDictに戻す
# hugging_face_dataset.reset_format()

csvファイルからDataFrameにしてデータをシャッフル後にDatasetDictにする

In [8]:
# csvファイルをDataFrameとして読み込み
train_df = pd.read_csv(filepath_or_buffer="./hugging_face_dataset_train.csv",
                       encoding="utf-8",
                       index_col=False)
validation_df = pd.read_csv(filepath_or_buffer="./hugging_face_dataset_validation.csv",
                            encoding="utf-8",
                            index_col=False)
test_df = pd.read_csv(filepath_or_buffer="./hugging_face_dataset_test.csv",
                      encoding="utf-8",
                      index_col=False)
# DataFrameの中身をシャッフルする
train_df = train_df.sample(frac=1).reset_index(drop=True)
validation_df = validation_df.sample(frac=1).reset_index(drop=True)
test_df = test_df.sample(frac=1).reset_index(drop=True)
# データ数を全体の5%のみにする
train_df = train_df.iloc[0:len(train_df)//20, :]
validation_df = validation_df.iloc[0:len(validation_df)//20, :]
test_df = test_df.iloc[0:len(test_df)//20, :]
# 事前学習済みBERTモデルで使えるようにDataFrameをDatasetへ変換
train_ds = datasets.Dataset.from_pandas(df=train_df)
validation_ds = datasets.Dataset.from_pandas(df=validation_df)
test_ds = datasets.Dataset.from_pandas(df=test_df)
# 変換したDatasetのlabel(正解ラベル)はint64の0,1,2になっているので、正解ラベルとしての0,1,2になるようにint64をClassLabelへ変換
class_labels = datasets.ClassLabel(num_classes=3,
                                   names=["positive", "neutral", "negative"])
train_ds = train_ds.cast_column(column="label",
                                feature=class_labels)
validation_ds = validation_ds.cast_column(column="label",
                                          feature=class_labels)
test_ds = test_ds.cast_column(column="label",
                              feature=class_labels)
# 各DatasetをDatasetDictで纏める
dataset_dict_from_csv = datasets.DatasetDict({"train": train_ds,
                                              "validation": validation_ds,
                                              "test": test_ds})
# DatasetDictを確認
print(dataset_dict_from_csv)
print(type(dataset_dict_from_csv))
# DatasetDictのデータの項目を確認(trainを対象にして確認)
pprint.pprint(dataset_dict_from_csv["train"].features)
# DatasetDictのtrainのデータの中身を2つ確認
pprint.pprint(dataset_dict_from_csv["train"][0:2])

Casting the dataset:   0%|          | 0/6000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/150 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/150 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 6000
    })
    validation: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 150
    })
    test: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 150
    })
})
<class 'datasets.dataset_dict.DatasetDict'>
{'label': ClassLabel(names=['positive', 'neutral', 'negative'], id=None),
 'source': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None)}
{'label': [0, 1],
 'source': ['amazon_reviews_multi', 'amazon_reviews_multi'],
 'text': ['家族から 無いとせがまれて クリックの翌日には手元に届き感謝しております。',
          '今までテレビの番組を録画すると、時々画像が乱れていたが、それがなくなったのでうまく機能しているので良かった。']}


**東北大BERTモデルで使われているトークナイザーを取得**

In [9]:
# 東北大BERTモデルのチェックポイント名
model_checkpoint = "cl-tohoku/bert-base-japanese-whole-word-masking"
# 東北大の事前学習済みBERTモデルのトークナイザーを取得
tokenizer = transformers.AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_checkpoint)
# 使っているトークナイザーの単語toIDデータ
pprint.pprint(tokenizer.get_vocab())

tokenizer_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/258k [00:00<?, ?B/s]

ストリーミング出力は最後の 5000 行に切り捨てられました。
 '発足': 2991,
 '発車': 11873,
 '発送': 22206,
 '発進': 14508,
 '発達': 4201,
 '発酵': 13119,
 '発電': 2389,
 '発音': 6231,
 '登': 370,
 '登っ': 19809,
 '登り': 18799,
 '登る': 20684,
 '登場': 656,
 '登山': 6982,
 '登板': 3101,
 '登校': 22861,
 '登用': 16880,
 '登記': 8976,
 '登録': 1800,
 '登頂': 20771,
 '白': 778,
 '白い': 6950,
 '白く': 21845,
 '白井': 17643,
 '白人': 7005,
 '白地': 26156,
 '白山': 12557,
 '白川': 17856,
 '白星': 25975,
 '白書': 26714,
 '白河': 11302,
 '白石': 12423,
 '白紙': 22722,
 '白色': 9686,
 '白血': 16581,
 '白血病': 20506,
 '白金': 23883,
 '白馬': 26715,
 '白鳥': 15204,
 '白黒': 22862,
 '百': 1625,
 '百姓': 18557,
 '百済': 24790,
 '百科': 14752,
 '百貨店': 9082,
 '的': 81,
 '的中': 23433,
 '的確': 25116,
 '皆': 5257,
 '皆殺し': 27843,
 '皆無': 18391,
 '皇': 1253,
 '皇位': 23725,
 '皇后': 7543,
 '皇太子': 9183,
 '皇女': 20155,
 '皇子': 11537,
 '皇室': 12625,
 '皇居': 21608,
 '皇帝': 2748,
 '皇族': 10145,
 '皐': 19219,
 '皐月': 20685,
 '皮': 3014,
 '皮切り': 10340,
 '皮肉': 13609,
 '皮膚': 7990,
 '皮質': 20772,
 '皿': 14318,
 '盃': 24728,
 '盆': 8262,
 '盆地': 110

In [ ]:
# 学習データのDatasetDictをトークン化する際に使う関数
def tokenize_datasetdict(batch):
    texts = [str(text) for text in batch["text"] if text is not None]  # text input must be of type str (single example), List[str] (batch or single pretokenized example) or List[List[str]を防ぐため
    print(type(texts), texts, len(texts), type(texts[0]), texts[0], sep="\n")
    return tokenizer(texts, padding=True, truncation=True, add_special_tokens=True, return_tensors="pt")

In [ ]:
# 学習データのDatasetDictをトークン化(map関数は、pandasのapply関数みたいなイメージ)
tokenized_dataset_dict = dataset_dict_from_csv.map(function=tokenize_datasetdict,
                                                   batched=True,
                                                   batch_size=None)

Parameter 'function'=<function tokenize_datasetdict at 0x7f1335274900> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

<class 'list'>
['家族から 無いとせがまれて クリックの翌日には手元に届き感謝しております。', '今までテレビの番組を録画すると、時々画像が乱れていたが、それがなくなったのでうまく機能しているので良かった。', '予想以上にページがすぐ取れてしまい、記録用に使っている時には正直困っています。 また紙質も昔アメリカで購入した同類商品と比べるとかなり落ちます。', 'その前に買ったバッテリーが回収品対象で買い替えました。 回収品のバッテリー途中でよく切れるので不良ふでしたが、このバッテリーでは問題なさそう。', '久々にセンチュリーを購入して、やはり今までで一番使いやすいと思う。 RAID1で使っていた他社製HDDケースが壊れたので購入して、危険を覚悟でRAID1にセットしてそのまま乗せたら使えました。（ただし、保証はないので絶対まねはしないでください）', '金属アレルギーは、出ない！ でも、石が2列の方は作りが雑で片方は石が取れてて使い物にならない。 せっかく可愛いのに残念でした。', 'バイクのシート張り替えに使用しました。安いけど十分役立ちました。ハンドタッカーとは比べ物にならないくらい楽に針もしっかり刺さります', 'このお値段には見えません、 実物を見たら期待以上でした。 非常に良いと思います。', '未だ落としていないので分かりませんが、置いた時の滑り落ちが、面を下にすると滑りにくいけど、背面素材&盛り上がり不足と背面スタンドで滑りやすい、又スタンドは、縦もつかえれば！', 'フィット感が気に入ってリピートしたら臭かったよ(泣)', '最初から使えませんでした。 本体に接続して、直ぐにライトが消え、全く充電出来ませんでした。 どうやら、外れに当たってしまったようです。', '購入して１ヶ月もせずに故障。ライトも点灯せず。購入検討の方は気を付けて。', '5年前くらいからG9口金のLEDは中国製しかないので不安で買わなかった。もういいだろうと思って20個買ってつかってみました。とてもいいです。お礼のメモや注意書きもパッケージに入っていて、ホスピタリティも高いです。', 'テレビでこの商品のいいところを拝見し, 購入したが, 1週間後地元のスーパーでも販売し始めたら,価格は半額以下だった。ここの販売価格は高すぎだと思う。', '梱包が悪く、酷

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

<class 'list'>
['PCに接続はできるのだが、ファイル等をコピー中に接続が自動的に否定されエラーが発生する', '1ヶ月しないうちに、タブレット用の端子で充電できなくなりました。一回こっきりで壊れるのを覚悟で買うなら問題ありません。', 'まじで最悪だよ。まずこんなので守れるのかってくらい薄いシリコンカバーに超もろそうなプラスチックみたいなやつがついてるだけ。一回落としてしまってそしたらバッキバキに割れたよ笑笑電話しても全く出ないし評価1もつけたくないレベル。中華製かな？', 'シンプルかつスリムです。作りはしっかりしていて、安っぽさは全くありません。', '初めて使用しました。シャンプーだけでサラサラになるのと、ツヤが出るのがすごいと思い、気に入りました。もう少し安く購入できるなら継続しやすいのに・・・', 'サム・ライミということでリストに入れていたと思うけれど、見たときにはすっかり忘れていました。 新しい映画だけあって映像はきれい。 実話とはとても思えない。つまらなくはないけれど面白いというほどではない。 残念ながら怖さは全然。', '何色でも合うように黒をチョイス！ 太めなので安定感があり思っていたよりかなり良いものでした。 以前より少し痩せたため、購入しなきゃいけなかったのですが。', '中国製品です。最初から結構な大きさの穴が空いていました。補修用のシールとして安っぽいテープが付いていましたが、もちろんそんなものでは防げません。この商品は全くお勧めできません。', '商品が着いて、袋を開けると強烈な灯油の臭いがした。試しに着てみたところ肌に痛みを感じた。 肌着として致命的だと思う。', '内容は楽しみですが、こんなに製本の出来の悪い文庫本は初めて見ました。 本の上面がノコギリみたいにギザギザです。', '1度目は、箱底面のシールが2か所とも切れた状態で届きました。 返品したところ、返金手続きのみでしたので、即日、改めて購入しましたが、到着した商品が、完全に開封された状態で届きました。前回の底面のシールに加え、「新品」と記載されたシールも開封痕があり、剥がれて居ましたので、箱が空けられる状態でしたので、中身を確認しましたが、説明書に変な折り目がついており、本来の折り方から考えると、逆に折っていましたので、表紙が内側に織り込まれている状態でした

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

<class 'list'>
['商品自体は途中から半割りになったファイバーが二本まとめて入力側端子に来ているという単純な物なので、入力が十分綺麗であれば役には立ちます ただ信号がかなり減退するようで、例えばパッシブ方式のセレクタからさらに分配、という使い方はムリ', '印刷クソ荒い、クソすぐ割れる、外すのクソ難しい クソみたいな買い物をしてしまいました', 'オートバイ用に購入。 肘と膝部分のプロテクターが弱い気がする、値段を考えると仕方ないかもしれませんが、オートバイや原付に乗る人には勧めません。 自転車やスケボーで転けたくらいなら問題ないと思います。', '昨日のお届けのはずですが届きません。 どこへ連絡すれば良いのでしょうか。', '今回革靴をネットショッピングで購入するのは初めてだったので、最初はサイズが合うかどうか心配してました。 普段履いている革靴が25.5cmなので、いつもと同じサイズを購入してみたところ、全く問題なく履くことが出来ました。 シンプルなデザインで、そのへんのショップで販売していても遜色ない型ですね。 色合いは明るすぎず、暗すぎない落ち着いたブラウンという感じ。 今のところ靴擦れはないので文句なしです。 見た目の高級感、フィット感、履き心地、光沢感どれもとても満足しています。 これからも大切に使用していきたいと思っています。', 'どんなに気をつけても途中で切れて途方にくれる。 急いでる時なんて最悪。 どんなに企業努力してもこのアナログさは変わらないのだと実感し、シリコンラップとジップロックに切り替えて快適です。', '個人的にデザインは気に入っています！ ですが、色々と造りが雑です。乗っているときなどに壊れなければ良いのですが。', '12月25日に注文。現在4月11日商品届かず、4個も買ったのに。', '基本的にアテレコではなく ナレーション風アテレコです。少し違和感はありますが 子供（低学年）程度なら楽しく見れますよ。', '7㎏のミニチュアシュナウザーには少し大きかった。ものはしっかりしているので、犬種によってはすごくいいとおもいますが、シュナウザー向きではないようでした。', '作品の時間を見ずに見始めたので「そろそろ終わりかな？」と思って残り時間を見たときに1時間近く残っていて驚愕しました。 全体の雰囲気は良かったのですが、

In [ ]:
# トークン化した学習データのDatasetDictを確認
print(tokenized_dataset_dict)
for feature in tokenized_dataset_dict["train"].features.keys():
    print("データ項目：{a}".format(a=feature))
    print(tokenized_dataset_dict["train"][feature][0])
ids_to_tokens = tokenizer.convert_ids_to_tokens(tokenized_dataset_dict["train"]["input_ids"][0])
print("データ項目：token")
print(ids_to_tokens)

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6000
    })
    validation: Dataset({
        features: ['text', 'source', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 150
    })
    test: Dataset({
        features: ['text', 'source', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 150
    })
})
データ項目：text
家族から 無いとせがまれて クリックの翌日には手元に届き感謝しております。
データ項目：source
amazon_reviews_multi
データ項目：label
0
データ項目：input_ids
[2, 2283, 40, 2825, 13, 191, 28458, 28491, 20, 16, 20718, 5, 4638, 7, 9, 27745, 7, 27969, 8906, 15, 16, 206, 2610, 8, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

**東北大BERTモデルを取得**

In [ ]:
# 東北大BERTモデルのチェックポイント名
model_checkpoint = "cl-tohoku/bert-base-japanese-whole-word-masking"
# 出力層のパーセプトロン数を、データのラベル数の形にして東北大BERTモデルを取得
n_class = tokenized_dataset_dict["train"].features["label"].num_classes
nn_model = transformers.AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_checkpoint,
                                                                           num_labels=n_class).to(DEVICE)

In [15]:
torchinfo.summary(nn_model,
                  input_size=(16, 512),
                  dtypes=[torch.int])
# dtypes(入力データの型)はデフォルトでfloatだけれど、自然言語処理では入力データのトークンはintなので、intを指定
# (参考URL：https://cocoinit23.com/huggingface-runtimeerror-failed-to-run-torchinfo/)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Layer (type:depth-idx)                                       Output Shape              Param #
BertForSequenceClassification                                [16, 3]                   --
├─BertModel: 1-1                                             [16, 768]                 --
│    └─BertEmbeddings: 2-1                                   [16, 512, 768]            --
│    │    └─Embedding: 3-1                                   [16, 512, 768]            24,576,000
│    │    └─Embedding: 3-2                                   [16, 512, 768]            1,536
│    │    └─Embedding: 3-3                                   [1, 512, 768]             393,216
│    │    └─LayerNorm: 3-4                                   [16, 512, 768]            1,536
│    │    └─Dropout: 3-5                                     [16, 512, 768]            --
│    └─BertEncoder: 2-2                                      [16, 512, 768]            --
│    │    └─ModuleList: 3-6                                  --             

**FineTuningを行う前での文章分類**

In [16]:
# 推論(テストデータを対象にして予測)
test_token_tensor = torch.LongTensor(np.array(tokenized_dataset_dict["test"]["input_ids"])).to(DEVICE)
test_attention_mask_tensor = torch.LongTensor(np.array(tokenized_dataset_dict["test"]["attention_mask"])).to(DEVICE)
with torch.no_grad():
    # https://huggingface.co/docs/transformers/v4.50.0/ja/model_doc/bert#transformers.BertForSequenceClassification.forward
    test_output_before = nn_model(input_ids=test_token_tensor,
                                  attention_mask=test_attention_mask_tensor)
test_output_before_array = test_output_before.logits.to("cpu").detach().numpy()  # 出力結果の属性logitsに行列もしくはテンソルがあって、それをcpu処理にして、gradを外して、numpyにする
test_predict_before = np.argmax(a=test_output_before_array,
                                axis=1)  # 予測したラベル
print("推論結果：", test_predict_before)
test_answer = np.array(tokenized_dataset_dict["test"]["label"])  # 正解ラベル
print("正解：", test_answer)
test_accuracy_result_before = metrics.accuracy_score(y_true=test_answer,
                                                     y_pred=test_predict_before)
print("Accuracy：", test_accuracy_result_before)

推論結果： [1 0 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1
 1 1 0 0 0 1 0 1 1 1 0 0 1 1 0 0 0 1 1 0 1 0 0 1 0 1 1 1 1 1 1 1 0 0 0 1 0
 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 0 0 1 1 0 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 0 1 0 1 0 0 0 0 0 1 1 1 0 0 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0
 0 0]
正解： [1 2 1 2 0 2 1 2 1 1 1 1 2 0 1 2 0 1 0 1 1 2 2 0 0 0 2 2 1 0 0 2 1 0 1 0 2
 0 1 2 0 0 1 1 2 0 1 1 0 2 1 1 0 1 1 2 1 2 1 1 0 2 0 1 1 0 0 0 0 1 1 0 2 2
 0 0 2 2 1 2 1 0 2 0 0 1 1 1 1 1 2 2 0 2 0 2 1 2 1 0 2 1 0 1 1 1 2 0 1 2 2
 2 1 0 1 0 1 1 2 1 2 2 0 2 2 2 0 1 2 0 2 2 1 1 0 0 0 1 0 0 0 0 2 2 2 1 0 1
 2 2]
Accuracy： 0.30666666666666664


**FineTuning**

In [17]:
# 評価関数を算出する関数(transformers.Trainerクラスをインスタンス化する時に使う)
def compute_metric(pred):  # 引数predにはEvalPredictionオブジェクトが入る
    answer = pred.label_ids  # EvalPredictionオブジェクト内のlabel_ids属性を取得(正解データの正解ラベルになる)
    predict = pred.predictions.argmax(-1)  # EvalPredictionオブジェクト内のprediction属性でargmax(-1)を取得(推測データの推測結果になる)
    precision_value, recall_value, f1_value, _ = metrics.precision_recall_fscore_support(y_true=answer,
                                                                                         y_pred=predict,
                                                                                         average="weighted")
    accuracy_value = metrics.accuracy_score(y_true=answer,
                                            y_pred=predict)
    return {"accuracy": accuracy_value,
            "recall": recall_value,
            "precision": precision_value,
            "f1": f1_value}

In [18]:
# 学習時のパラメータを設定(transformers.Trainerクラスをインスタンス化する時に使う)
minibatch_size = 32
training_args = transformers.TrainingArguments(output_dir=".",  # ファイルの出力先
                                               run_name="training_for_BERT_finetuning",
                                               do_train=True,
                                               do_eval=True,
                                               eval_strategy="epoch",  # 評価関数を計算するタイミング
                                               logging_steps=tokenized_dataset_dict["train"].num_rows//minibatch_size,  # ログを出すタイミング
                                               log_level="error",  # ログレベル
                                               save_strategy="no",  # チェックポイントを保存するタイミング
                                               save_total_limit=None,  # チェックポイントを保存する数
                                               label_names=["label"],  # 正解ラベルのカラム名
                                               learning_rate=2e-5,  # 学習率
                                               weight_decay=0.01,  # 減衰率
                                               per_device_train_batch_size=minibatch_size,  # 学習の時のミニバッチ数
                                               per_device_eval_batch_size=minibatch_size,  # 検証の時のミニバッチ数
                                               fp16=True,  # 16ビット計算
                                               num_train_epochs=3,  # 学習エポック数
                                               report_to="none")  # trainメソッド時のYou must call wandb.init() before wandb.log()を防ぐため

In [19]:
# 学習のクラスでインスタンスを作成
nn_model_trainer = transformers.Trainer(model=nn_model,
                                        args=training_args,
                                        compute_metrics=compute_metric,
                                        train_dataset=tokenized_dataset_dict["train"],
                                        eval_dataset=tokenized_dataset_dict["validation"],
                                        processing_class=tokenizer)

In [20]:
# trainメソッドでHuggingFaceのAPIキーを求められないように事前にログイン
login_success = login(token=HuggingFace_API_KEY)

In [21]:
# 学習
nn_model_trainer.train()

Epoch,Training Loss,Validation Loss
1,0.679400,No log
2,0.447800,No log
3,0.326100,No log


TrainOutput(global_step=564, training_loss=0.48264366388320923, metrics={'train_runtime': 404.7929, 'train_samples_per_second': 44.467, 'train_steps_per_second': 1.393, 'total_flos': 4736041519104000.0, 'train_loss': 0.48264366388320923, 'epoch': 3.0})

In [22]:
# 検証データ(validデータ)でのaccuracyを確認
valid_output = nn_model_trainer.predict(test_dataset=tokenized_dataset_dict["validation"])  # validデータで推論
valid_predict = np.argmax(a=valid_output.predictions[1],
                          axis=1)  # 出力結果のpredictions属性のインデックス[1]に(時系列数, ラベル数)の行列がある
print("推論結果：", valid_predict)
valid_answer = np.array(tokenized_dataset_dict["validation"]["label"])
print("正解：", valid_answer)
valid_accuracy_result = metrics.accuracy_score(y_true=valid_answer,
                                               y_pred=valid_predict)
print("Accuracy：", valid_accuracy_result)

推論結果： [2 2 2 0 0 2 0 2 2 2 2 0 2 0 2 2 1 2 1 2 0 0 0 0 2 2 2 0 1 0 2 1 0 2 1 0 2
 1 1 0 1 0 2 0 1 1 0 1 0 0 2 0 0 2 1 1 1 2 2 1 1 1 1 2 2 0 0 0 2 0 1 2 0 0
 2 1 1 2 2 2 2 2 0 1 0 1 0 2 1 2 0 2 2 0 1 2 2 0 2 1 1 0 0 2 0 1 1 0 1 1 2
 2 2 2 1 2 0 0 0 1 2 0 0 1 0 1 1 1 0 0 2 1 0 1 2 0 2 1 2 2 1 0 2 2 1 0 2 2
 1 0]
正解： [2 2 2 0 0 1 0 2 2 2 2 0 2 0 2 2 1 2 1 2 0 0 0 0 2 2 2 0 0 0 2 1 0 2 1 0 2
 1 1 0 1 1 2 0 1 2 0 2 0 0 2 0 0 2 1 0 0 2 2 1 1 1 2 2 1 1 0 0 2 0 1 2 0 0
 2 1 1 2 2 2 2 2 1 0 0 0 0 2 0 2 0 2 2 0 1 1 2 0 2 1 1 1 0 2 0 1 1 0 1 2 1
 2 2 1 1 2 1 0 0 1 2 0 0 1 0 1 1 1 0 1 2 0 0 1 2 0 2 0 2 2 2 0 2 2 2 0 2 2
 1 0]
Accuracy： 0.8333333333333334


In [23]:
# 学習(Fine Tuning)したモデルを保存
id2value = {}  # 分類対象のカラム(目的変数)のIDtoVALUEを保存
value2id = {}  # 分類対象のカラム(目的変数)のVALUEtoIDを保存
for i in range(tokenized_dataset_dict["train"].features["label"].num_classes):
    id2value[i] = tokenized_dataset_dict["train"].features["label"].int2str(i)
    value2id[tokenized_dataset_dict["train"].features["label"].int2str(i)] = i
print("id2value：", id2value, sep="\n")
nn_model_trainer.model.config.id2label = id2value
print("value2id：", value2id, sep="\n")
nn_model_trainer.model.config.label2id = value2id
nn_model_trainer.save_model(output_dir=".")
nn_model_trainer.save_state()

id2value：
{0: 'positive', 1: 'neutral', 2: 'negative'}
value2id：
{'positive': 0, 'neutral': 1, 'negative': 2}


**FineTuningを行った後での文章分類**

In [24]:
# 学習(Fine Tuning)したモデルをロード
fine_tuning_tokenizer = transformers.AutoTokenizer.from_pretrained(pretrained_model_name_or_path=".")
fine_tuning_nn_model = transformers.AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path=".").to(DEVICE)

In [25]:
# 推論(テストデータを対象にして予測)
test_token_tensor = torch.LongTensor(np.array(tokenized_dataset_dict["test"]["input_ids"])).to(DEVICE)
test_attention_mask_tensor = torch.LongTensor(np.array(tokenized_dataset_dict["test"]["attention_mask"])).to(DEVICE)
with torch.no_grad():
    # https://huggingface.co/docs/transformers/v4.50.0/ja/model_doc/bert#transformers.BertForSequenceClassification.forward
    test_output = fine_tuning_nn_model(input_ids=test_token_tensor,
                                       attention_mask=test_attention_mask_tensor)
test_output_array = test_output.logits.to("cpu").detach().numpy()  # 出力結果の属性logitsに行列もしくはテンソルがあって、それをcpu処理にして、gradを外して、numpyにする
test_predict = np.argmax(a=test_output_array,
                         axis=1)  # 予測したラベル
print("推論結果：", test_predict)
test_answer = np.array(tokenized_dataset_dict["test"]["label"])  # 正解ラベル
print("正解：", test_answer)
test_accuracy_result = metrics.accuracy_score(y_true=test_answer,
                                              y_pred=test_predict)
print("Accuracy：", test_accuracy_result)

推論結果： [1 2 1 2 0 1 1 2 1 1 1 0 2 0 2 2 0 1 0 1 1 2 2 0 1 2 1 1 1 0 0 1 1 0 1 1 2
 0 0 2 0 0 1 2 1 0 1 1 0 2 0 2 0 2 1 2 1 2 1 1 1 2 0 1 1 0 0 0 0 1 1 0 2 2
 0 1 2 2 1 2 0 0 2 0 0 0 1 2 1 1 2 0 0 2 0 2 1 1 2 0 2 1 0 1 1 1 2 0 1 2 2
 2 2 0 1 1 1 1 2 1 1 2 0 2 2 1 0 2 2 0 2 2 1 1 0 0 0 1 0 0 2 0 2 2 2 1 0 1
 2 2]
正解： [1 2 1 2 0 2 1 2 1 1 1 1 2 0 1 2 0 1 0 1 1 2 2 0 0 0 2 2 1 0 0 2 1 0 1 0 2
 0 1 2 0 0 1 1 2 0 1 1 0 2 1 1 0 1 1 2 1 2 1 1 0 2 0 1 1 0 0 0 0 1 1 0 2 2
 0 0 2 2 1 2 1 0 2 0 0 1 1 1 1 1 2 2 0 2 0 2 1 2 1 0 2 1 0 1 1 1 2 0 1 2 2
 2 1 0 1 0 1 1 2 1 2 2 0 2 2 2 0 1 2 0 2 2 1 1 0 0 0 1 0 0 0 0 2 2 2 1 0 1
 2 2]
Accuracy： 0.8066666666666666
